[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/sdxl-colab/blob/main/sdxl_v0.9_comfyui_colab.ipynb)

In [ ]:
import os

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 -U
!pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf

!git clone -b v1.3 https://github.com/camenduru/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -d /content/ComfyUI/models/upscale_models -o RealESRGAN_x2plus.pth

base = "https://dagshub.com/sdxl/stable-diffusion-xl-base-0.9/raw/d82c04e3c1866d98718a4c0f7d91392bcd3e0bcd/data/sd_xl_base_0.9.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {base} -d /content/ComfyUI/models/checkpoints -o sd_xl_base_0.9.safetensors

refiner = "https://dagshub.com/sdxl/stable-diffusion-xl-refiner-0.9/raw/e7aa1d4794e5c84e1f3940ff082a61be751432ef/data/sd_xl_refiner_0.9.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {refiner} -d /content/ComfyUI/models/checkpoints -o sd_xl_refiner_0.9.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -d /content/ComfyUI/models/vae -o sdxl_vae.vae.safetensors

!python main.py --dont-print-server